In [8]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from google.colab import files

project = 'pnad-covid-460322' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

# CONSULTA PNAD

In [13]:
job = client.get_job('bquxjob_2d2cafce_196fafdca4d')
destination = job.destination
bq_df = bpd.read_gbq(f"{destination.project}.{destination.dataset_id}.{destination.table_id}")
pandas_df = bq_df.to_pandas()

pandas_df

,UF,mes_pesquisa,idade,sexo,escolaridade,recebeu_auxilio,teve_febre_semana_passada,teve_tosse_semana_passada,teve_dor_garganta_semana_passada,teve_dificuldade_respirar,...,teve_nausea,teve_nariz_entupido,trabalhou_semana_passada,fez_exame,resultado_swab,resultado_furo_no_dedo,resultado_veia_no_braco,providencia_exame,resultado_final,total
0,21,9,53,1,5,2,2,2,2,1,...,2,2,2.0,1,<NA>,2.0,<NA>,3,Negativo,1
1,28,7,37,1,3,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,1,<NA>,1
2,29,8,82,1,2,2,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,3,<NA>,4
3,35,7,54,1,2,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,2
4,29,8,7,2,1,1,2,2,2,2,...,2,2,<NA>,2,<NA>,<NA>,<NA>,4,<NA>,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473238,21,7,24,1,4,2,2,2,2,2,...,2,2,1.0,2,<NA>,<NA>,<NA>,2,<NA>,1
473239,26,9,19,2,4,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,1
473240,43,7,21,1,2,1,2,2,2,2,...,2,2,1.0,2,<NA>,<NA>,<NA>,3,<NA>,1
473241,53,7,70,2,3,2,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,2


# DICIONÁRIO DE RESPOSTA


In [14]:
job_dict = client.get_job('bquxjob_1e946717_196fafa3d44')
destination_dict = job_dict.destination
bq_dict_df = bpd.read_gbq(f"{destination_dict.project}.{destination_dict.dataset_id}.{destination_dict.table_id}")
pandas_dict_df = bq_dict_df.to_pandas()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2025-05-23 02:30:37.166708+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


# TRATAMENTO DE RESPOSTAS

In [15]:
df_query = pandas_df.copy()

dict_map = {}
for question in pandas_dict_df['Question'].unique():
    sub_df = pandas_dict_df[pandas_dict_df['Question'] == question]
    mapping = dict(zip(sub_df['Value'], sub_df['Ref']))
    dict_map[question] = mapping

# Agora aplicamos esse mapeamento para cada coluna da query:
for col in df_query.columns:
    if col in dict_map:
        df_query[col] = df_query[col].map(dict_map[col])




### **ANÁLISE TEMPORAL**
* Quantidade de testes
* Avaliar tipo de teste
* Providencias tomadas
* Trabalhou na semana anterior
* Recebeu auxílio

In [16]:
df_query_temp = df_query.groupby(['mes_pesquisa'], as_index=False).agg({'total':'sum'})
df_query_temp = df_query_temp.pivot_table(index='mes_pesquisa', values='total')


df_query_temp_exam = df_query.groupby(['mes_pesquisa', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_temp_exam = df_query_temp_exam.pivot_table(index='mes_pesquisa', columns='fez_exame', values='total')

df_query_temp_result = df_query.groupby(['mes_pesquisa', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_temp_result = df_query_temp_result.pivot_table(index='mes_pesquisa', columns='resultado_final', values='total')


df_query_temp_trab = df_query.groupby(['mes_pesquisa', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_temp_trab = df_query_temp_trab.pivot_table(index='mes_pesquisa', columns='trabalhou_semana_passada', values='total')

df_query_temp_aux = df_query.groupby(['mes_pesquisa', 'recebeu_auxilio'], as_index=False).agg({'total':'sum'})
df_query_temp_aux = df_query_temp_aux.pivot_table(index='mes_pesquisa', columns='recebeu_auxilio', values='total')

with pd.ExcelWriter("Temporal.xlsx", engine="openpyxl") as writer:
    df_query_temp.to_excel(writer, sheet_name="Total por Mês")
    df_query_temp_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_temp_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_temp_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_temp_aux.to_excel(writer, sheet_name="Auxílio")

# files.download('Temporal.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>